In [3]:
import pandas as pd
import numpy as np

In [5]:
# The code was removed by Watson Studio for sharing.

,key,Postal code,Borough,Neighbourhood
0,1,M1A,Not assigned,Not assigned
1,2,M2A,Not assigned,Not assigned
2,3,M3A,North York,Parkwoods
3,4,M4A,North York,Victoria Village
4,5,M5A,Downtown Toronto,Harbourfront


In [6]:
df_data_0.drop(df_data_0.loc[df_data_0['Borough']=="Not assigned"].index, inplace=True)
df_data_0.drop(['key'], axis=1, inplace=True)

In [7]:
df1=df_data_0.groupby(df_data_0['Postal code'])['Neighbourhood'].apply(', '.join).reset_index()
df1.head()

,Postal code,Neighbourhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [8]:
df2=df_data_0.groupby(df_data_0['Postal code'])['Borough'].apply('_'.join).reset_index()
df2.head()

,Postal code,Borough
0,M1B,Scarborough_Scarborough
1,M1C,Scarborough_Scarborough_Scarborough
2,M1E,Scarborough_Scarborough_Scarborough
3,M1G,Scarborough
4,M1H,Scarborough


In [9]:
df3=pd.merge(df2, df1, on="Postal code")
df3["Borough"]= df3["Borough"].str.split("_", n = -1, expand = True)
df3['Neighbourhood'][93]="Queen's Park"
df3.head()

,Postal code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:

body = client_6a7f86d3cb41458a83661c5367b17e1f.get_object(Bucket='capstoneproject-donotdelete-pr-9umdxa0rwswwbg',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df4=df_data_1

In [16]:
df4.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df4.rename(columns={'Postal Code':'Postal code'}, inplace=True)
df4.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df5=pd.merge(df3, df4, on="Postal code")
df5.head()

,Postal code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
